# Import modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import copy
import pytz
import datetime 
import cPickle as pickle
import astral
import astropy.time as aptime
from scipy import stats
from scipy import interpolate

# Load Data

In [ ]:
csv_file1 = pd.read_csv('IFS_RawContrast_Gemini_env_20180530_all_seeing.csv')  # final contrast data
csv_file1_copy = copy.copy(csv_file1)
raw_contrast_data= pd.DataFrame(csv_file1_copy)
raw_contrast_data['dts']=np.nan

csv_file2 = pd.read_csv('Final_HSpec_Contrasts_20170828.txt')  # final contrast data
csv_file2_copy = copy.copy(csv_file2)
final_contrast_data= pd.DataFrame(csv_file2_copy)

#  Create colums with datetimes and make them timezone aware
raw_contrast_data['dts']= aptime.Time(raw_contrast_data['MJDOBS'], format='mjd').datetime 
raw_contrast_data['dts']=raw_contrast_data['dts'].dt.tz_localize(pytz.UTC)

final_contrast_data['dts'] = pd.to_datetime(final_contrast_data['datestr'].values,yearfirst = True, format = '%y%m%d')
final_contrast_data=final_contrast_data.rename(columns = {'object':'OBJNAME', 'datestr':'DATESTR','mode':'OBSMODE'})

# Scale contrast with fov

In [ ]:
cond = final_contrast_data['fov_rot'] > 12.2  # calculated min angle  
final_contrast_data = final_contrast_data.loc[cond]
final_contrast_data = final_contrast_data.set_index(np.arange(len(final_contrast_data)))

Solve for differences

In [ ]:
raw_contrast_data['OMSA_delT'] = raw_contrast_data['OMSATEMP'] - raw_contrast_data['TAMBIENT']
raw_contrast_data['M1_avg_delT'] = raw_contrast_data['M1_avg'] - raw_contrast_data['TAMBIENT']
raw_contrast_data['OE_Interior_Air_temperature_delT'] = raw_contrast_data['OE_Interior_Air_temperature'] - raw_contrast_data['TAMBIENT']

Group the data by object and by dates 

In [ ]:
grouped = raw_contrast_data.groupby(['OBJNAME','DATESTR'],as_index=False).mean()

In [ ]:
c = raw_contrast_data.groupby(['OBJNAME','DATESTR'],as_index=False).size().to_frame(name='counts')

In [ ]:
grouped = grouped.join(c,on = ['OBJNAME','DATESTR'],how = 'inner')

merge the grouped data with teh final contrast data

In [ ]:
merge_data = pd.merge(final_contrast_data,grouped, on = ['OBJNAME','DATESTR'])

In [ ]:
merge_data['C40_CH4_scaled'] = merge_data['C40_CH4'] * np.sqrt(merge_data['counts']/38.) 

In [ ]:
print len(pd.unique(merge_data['OBJNAME']))

# Save data table as CSV file

In [ ]:
merge_data.to_csv('IFS_FinalContrast_20180530.csv',index=False)